In [1]:
import obspy
from obspy import read
import numpy as np
import os

In [2]:
#Make folder for AI ready data
! mkdir data/ai_ready
filepath = os.getcwd() + '/data/ai_ready/'
clean_filepath = os.getcwd() + '/data/clean/'

mkdir: data/ai_ready: File exists


In [3]:
#read in cleaned mseed files, save as daily npy files and one giant 100 day file
hundred_data = np.array([])

for i in range(100):
    i += 1

    #read mseed
    st = read(clean_filepath+str(i)+'_cleaned.mseed')

    #save data to day long npys
    data = st[0].data[0:-1] #removing extra point from decimating
    np.save(filepath+str(i)+'_ready.npy', data)

    #append to overall array
    hundred_data = np.append(hundred_data, data)

#save hundred day data npy
np.save(filepath+'100_days.npy', hundred_data)

KeyboardInterrupt: 